In [1]:
# !pip install datasets
# !pip install transformers

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Use a pipeline as a high-level helper
from transformers import pipeline

# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("NepBERTa/NepBERTa")
model = AutoModelForSequenceClassification.from_pretrained("NepBERTa/NepBERTa",from_tf=True).to(device)

classifier = pipeline("sentiment-analysis", model=model, tokenizer = tokenizer)

cuda


All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [4]:
import pandas as pd

df_train = pd.read_csv("/content/drive/My Drive/NepBERTa/train.csv")
df_test = pd.read_csv("/content/drive/My Drive/NepBERTa/test.csv")
main_path = "/content/drive/My Drive/NepBERTa/"

In [5]:
df_train

,text,label
0,चीनले दक्षिण एसियाली मुलुकहरु पाकिस्तान अफगानि...,2
1,हजुर यो कुरा देश सबै जनता ले सुननु परछ अब हुन...,1
2,अब भने कोभिड समस्या हल हुने भयो नेपाल सरकार को...,1
3,रबि जि तपाईं ले गरेको काम देखदा जो कोहिलाइ पनि...,1
4,पुरुष भनेर ठाडो शिर बनाएर हिड्नु नि गार्हो,0
...,...,...
35014,वायु प्रदूषणले संक्रमणको थप जोखिम कोभिड बाट मृ...,0
35015,गण्डकी प्रदेशका प्रहरी प्रमुख डीआइजीमा कोभिड स...,0
35016,"आउलास भोट माग्न भ्रस्टचारी , 34:31 बाट भ्रस्टच...",2
35017,हाम्रो नेपाली समाजमा /संबिधानमा यस्तो कानुन आउ...,2


In [6]:
df_test

,text,label
0,अपाङ्गता भएका बालबालिकालाई कोभिड पछि वैकल्पिक ...,0
1,सूरदास के इस भजन में विश्व के बर्तमान स्थिति ज...,0
2,हन चसमा कहिले पनि देखेको,2
3,सन्दर्भ कोभिड् कोरोना,1
4,नेपाल प्रहरीको मुख्यालय प्रहरी प्रधान कार्यालय...,0
...,...,...
8750,ईद उल फितर पर्वले शान्ति र एकता कायम गर्दै कोभ...,1
8751,कोभिड बारे सूचना सामग्री समेटिएको पुस्तक उद्यो...,2
8752,हिन्दु महिलाहरूको महान चाड हरितालिका तीजको उपल...,1
8753,भान्सा कि भान्छा?,2


In [7]:
df_train.dtypes, df_test.dtypes

(text     object
 label     int64
 dtype: object,
 text     object
 label     int64
 dtype: object)

In [8]:
df_train['label'].value_counts()

1    15333
0    13889
2     5797
Name: label, dtype: int64

In [9]:
df_test['label'].value_counts()

1    3813
0    3506
2    1436
Name: label, dtype: int64

In [10]:
# For Train
df_train.drop(df_train[df_train['label'] == '-'].index, inplace = True)
df_train.drop(df_train[df_train['label'] == '20'].index, inplace = True)
df_train.drop(df_train[df_train['label'] == '11'].index, inplace = True)
df_train.drop(df_train[df_train['label'] == 'o'].index, inplace = True)
df_train.drop(df_train[df_train['label'] == '--'].index, inplace = True)

# For Test
df_test.drop(df_test[df_test['label'] == '-'].index, inplace = True)
df_test.drop(df_test[df_test['label'] == 'o'].index, inplace = True)

In [11]:
df_train['label'].value_counts()

1    15333
0    13889
2     5797
Name: label, dtype: int64

In [12]:
df_test['label'].value_counts()

1    3813
0    3506
2    1436
Name: label, dtype: int64

In [13]:
# label_mapping = {'0':0, '1':1, '2':2}

# df_train['label'] = df_train['label'].map(label_mapping)
# df_test['label'] = df_test['label'].map(label_mapping)

# df_train.dtypes, df_test.dtypes

In [14]:
df_train.isna().sum(), df_test.isna().sum()

(text     2
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64)

In [15]:
df_train = df_train.dropna()
df_test = df_test.dropna()
df_train.isna().sum(), df_test.isna().sum()

(text     0
 label    0
 dtype: int64,
 text     0
 label    0
 dtype: int64)

In [16]:
df_train.head()

,text,label
0,चीनले दक्षिण एसियाली मुलुकहरु पाकिस्तान अफगानि...,2
1,हजुर यो कुरा देश सबै जनता ले सुननु परछ अब हुन...,1
2,अब भने कोभिड समस्या हल हुने भयो नेपाल सरकार को...,1
3,रबि जि तपाईं ले गरेको काम देखदा जो कोहिलाइ पनि...,1
4,पुरुष भनेर ठाडो शिर बनाएर हिड्नु नि गार्हो,0


In [17]:
df_test.sample(5)

,text,label
4090,कोभिड को महामारीबाट मुक्त हुन र सुरक्षित जीवनय...,1
8719,खुकुलो यात्राको एउटा आधार सायद कोभिड को खोप हु...,1
7870,कुरा त सहि हो तर यही कोभिड ले गाह्रो पर्छ होला...,0
3434,काठमाडौंले कोभिड भुलेजस्तै म पनि तिमीलाई भुल्न...,2
5383,आइतबार आकस्मिक र कोभिड बाहेकका सम्पूर्ण सेवा ब...,0


In [18]:
from datasets import Dataset, DatasetDict

df_trainset = Dataset.from_pandas(df_train)
test_dataset = Dataset.from_pandas(df_test)

final_dataset = DatasetDict({
    'train':df_trainset,
    'test': test_dataset,
})

In [19]:
def tokenize_texts(batch):
    return tokenizer(batch['text'], truncation=True, padding='max_length', max_length=128, return_tensors='pt')

In [20]:
encoded_data = final_dataset.map(tokenize_texts, batched=True, batch_size=None)

Map:   0%|          | 0/35017 [00:00<?, ? examples/s]

Map:   0%|          | 0/8755 [00:00<?, ? examples/s]

In [21]:
encoded_data

DatasetDict({
    train: Dataset({
        features: ['text', 'label', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 35017
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8755
    })
})

In [22]:
def extract_hidden_states(batch):
  inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
    # Extract last hidden states
  with torch.no_grad():
      last_hidden_state = model(**inputs).logits
          # Return vector for [CLS] token
      return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [23]:
encoded_data.set_format("torch",columns=['input_ids', "attention_mask", "label"])

In [24]:
text = "असाध्यै राम्रो कार्यक्रम आयोजना गरिएको छ."

# Tokenize the text
inputs = tokenizer(text, return_tensors="pt").to(device)

# Forward pass through the model
outputs = model(**inputs)

# Access the last hidden states
last_hidden_states = outputs.logits

In [25]:
encoded_data = encoded_data.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/35017 [00:00<?, ? examples/s]

Map:   0%|          | 0/8755 [00:00<?, ? examples/s]

In [26]:
from transformers import AutoModelForSequenceClassification
num_labels = 3
model = (AutoModelForSequenceClassification
         .from_pretrained("NepBERTa/NepBERTa", num_labels=num_labels, from_tf=True)
         .to(device))

All TF 2.0 model weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceClassification were initialized from the TF 2.0 model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertForSequenceClassification for predictions without further training.


In [27]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [28]:
# !pip install "accelerate>=0.16.0,<1" "transformers[torch]>=4.28.1,<5" "torch>=1.13.1,<2"

In [29]:
from transformers import Trainer, TrainingArguments
batch_size = 64
logging_steps = len(encoded_data["train"])
model_name = main_path+"/model"
training_args = TrainingArguments(output_dir='/content/drive/MyDrive/NepBERTa',
                                  num_train_epochs=5,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

In [ ]:
trainer = Trainer(model=model, args=training_args,
                      compute_metrics=compute_metrics,train_dataset=encoded_data["train"],
                      eval_dataset=encoded_data["test"],tokenizer=tokenizer)
trainer.train();

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.634155,0.740491,0.731708
2,No log,0.648131,0.745517,0.725828
3,No log,0.650126,0.755111,0.750750
4,No log,0.693719,0.755340,0.751503
5,No log,0.724832,0.753969,0.749963


In [ ]:
preds_output = trainer.predict(encoded_data["test"])

In [ ]:
preds_output.metrics

{'test_loss': 0.72483229637146,
 'test_accuracy': 0.7539691604797258,
 'test_f1': 0.7499630271163834,
 'test_runtime': 60.6886,
 'test_samples_per_second': 144.261,
 'test_steps_per_second': 2.257}

In [ ]:
trainer.save_model("/content/drive/MyDrive/NepBERTa/saved_model")

In [ ]:
custom_text = "म तिमिलाइ राम्रो साथि मान्छु"
pipe = pipeline("text-classification", model='/content/drive/MyDrive/NepBERTa/saved_model')
preds = pipe(custom_text)

In [ ]:
preds

[{'label': 'LABEL_1', 'score': 0.972939133644104}]